In [1]:
import numpy as np
import pandas as pd

# 基础用法

In [2]:
# 创建基础数据
index = pd.date_range('2020/01/01', periods=8)
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])
df = pd.DataFrame(np.random.randn(8, 3), index=index, columns=['A', 'B', 'C'])
df

,A,B,C
2020-01-01,0.183959,0.804912,-0.870006
2020-01-02,0.786866,1.500421,-0.691150
2020-01-03,-0.894035,-1.563827,-1.223592
2020-01-04,-0.031196,1.325788,2.855762
2020-01-05,-1.660210,0.701103,-0.467863
2020-01-06,0.721188,1.163131,-1.002405
2020-01-07,-2.149763,0.962766,1.482660
2020-01-08,-0.490471,-2.028747,-0.476700


## Head与Tail

In [3]:
# head()与 tail()用于快速预览 Series 与 DataFrame，默认显示 5 条数据，也可以指定显示数据的数量。
long_series = pd.Series(np.random.randn(1000))
long_series.head()

0   -0.195522
1    0.170927
2    0.785614
3    0.806038
4    0.431143
dtype: float64

In [4]:
long_series.tail(3)

997    1.080037
998   -0.848246
999    1.042575
dtype: float64

## 属性与底层数据

Pandas 可以通过多个属性访问元数据：
- shape
    - 输出对象的轴维度，与 ndarray 一致
- 轴标签
    - Series: Index (仅有此轴)
    - DataFrame: Index (行) 与列

注意： **为属性赋值是安全的！**

In [5]:
df[:2]

,A,B,C
2020-01-01,0.183959,0.804912,-0.870006
2020-01-02,0.786866,1.500421,-0.691150


In [6]:
df.columns = [x.lower() for x in df.columns]
df

,a,b,c
2020-01-01,0.183959,0.804912,-0.870006
2020-01-02,0.786866,1.500421,-0.691150
2020-01-03,-0.894035,-1.563827,-1.223592
2020-01-04,-0.031196,1.325788,2.855762
2020-01-05,-1.660210,0.701103,-0.467863
2020-01-06,0.721188,1.163131,-1.002405
2020-01-07,-2.149763,0.962766,1.482660
2020-01-08,-0.490471,-2.028747,-0.476700


Pandas 对象，Series，DataFrame相当于数组的容器，用于存储数据、执行计算。大部分类型的底层数组都是 numpy.ndarray。
不过，Pandas 与第三方支持库一般都会扩展 NumPy 类型系统，添加自定义数组。

.array 属性用于提取 Index (opens new window)或 Series (opens new window)里的数据。

In [7]:
s.array

<PandasArray>
[-0.5751327481808248, -0.1831274261836204, -1.3016538521022516,
  -0.523955627981821, -0.5182583583346914]
Length: 5, dtype: float64

In [8]:
s.index.array

<PandasArray>
['a', 'b', 'c', 'd', 'e']
Length: 5, dtype: object

array一般指 ExtensionArray。至于什么是 ExtensionArray及 Pandas 为什么要用 ExtensionArray不是本节要说明的内容。更多信息请参阅数据类型。

提取 NumPy 数组，用 to_numpy()或 numpy.asarray()。

In [9]:
s.to_numpy()

array([-0.57513275, -0.18312743, -1.30165385, -0.52395563, -0.51825836])

In [10]:
np.asarray(s)

array([-0.57513275, -0.18312743, -1.30165385, -0.52395563, -0.51825836])

Series 与 Index 的类型是 ExtensionArray时， to_numpy()会复制数据，并强制转换值。详情见数据类型。

to_numpy()可以控制 numpy.ndarray生成的数据类型。以带时区的 datetime 为例，NumPy 未提供时区信息的 datetime 数据类型，Pandas 则提供了两种表现形式：

1. 一种是带 Timestamp的 numpy.ndarray，提供了正确的 tz 信息。

2. 另一种是 datetime64，这也是一种 numpy.ndarray，值被转换为 UTC，但去掉了时区信息。

时区信息可以用 dtype=object 保存。

In [11]:
ser = pd.Series(pd.date_range('2000', periods=2, tz="CET"))
ser

0   2000-01-01 00:00:00+01:00
1   2000-01-02 00:00:00+01:00
dtype: datetime64[ns, CET]

In [12]:
ser.to_numpy(dtype=object)

array([Timestamp('2000-01-01 00:00:00+0100', tz='CET', freq='D'),
       Timestamp('2000-01-02 00:00:00+0100', tz='CET', freq='D')],
      dtype=object)

或用datetime64[ns]去除时间戳,转换为UTC时间

In [13]:
# 或用datetime64[ns]去除时间戳,转换为UTC时间
ser.to_numpy(dtype="datetime64[ns]")

array(['1999-12-31T23:00:00.000000000', '2000-01-01T23:00:00.000000000'],
      dtype='datetime64[ns]')

提取 DataFrame 里的原数据稍微有点复杂。DataFrame 里所有列的数据类型都一样时，DataFrame.to_numpy()返回底层数据：



In [14]:
df.to_numpy()

array([[ 0.18395923,  0.80491156, -0.87000589],
       [ 0.78686618,  1.50042125, -0.69114985],
       [-0.89403477, -1.56382742, -1.22359218],
       [-0.0311957 ,  1.32578752,  2.85576195],
       [-1.66021003,  0.70110288, -0.467863  ],
       [ 0.72118782,  1.16313074, -1.00240457],
       [-2.14976342,  0.96276562,  1.48266037],
       [-0.4904712 , -2.02874722, -0.47669983]])

DataFrame 为同构型数据时，Pandas 直接修改原始 ndarray，所做修改会直接反应在数据结构里。对于异质型数据，即 DataFrame 列的数据类型不一样时，就不是这种操作模式了。与轴标签不同，不能为值的属性赋值。

> 注意:  
DataFrame 为同构型数据时，Pandas 直接修改原始 ndarray，所做修改会直接反应在数据结构里。对于异质型数据，即 DataFrame 列的数据类型不一样时，就不是这种操作模式了。与轴标签不同，不能为值的属性赋值。

以前，Pandas 推荐用 Series.values或 DataFrame.values从 Series 或 DataFrame 里提取数据。旧有代码库或在线教程里仍在用这种操作，但 Pandas 已改进了此功能，现在，推荐用 .array 或 to_numpy 提取数据，别再用 .values 了。.values 有以下几个缺点：
1. Series 含扩展类型时，Series.values无法判断到底是该返回 NumPy array，还是返回 ExtensionArray。而 Series.array则只返回 ExtensionArray，且不会复制数据。Series.to_numpy()则返回 NumPy 数组，其代价是需要复制、并强制转换数据的值。
2. DataFrame 含多种数据类型时，DataFrame.values会复制数据，并将数据的值强制转换同一种数据类型，这是一种代价较高的操作。DataFrame.to_numpy()则返回 NumPy 数组，这种方式更清晰，也不会把 DataFrame 里的数据都当作一种类型。

## 加速操作

借助 numexpr 与 bottleneck 支持库，Pandas 可以加速特定类型的二进制数值与布尔操作。

处理大型数据集时，这两个支持库特别有用，加速效果也非常明显。 numexpr 使用智能分块、缓存与多核技术。bottleneck 是一组专属 cython 例程，处理含 nans 值的数组时，特别快。

请看下面这个例子（DataFrame 包含 100 列 X 10 万行数据）:  


| 操作 | 0.11.0版 (ms) | 旧版 (ms) | 提升比率 |
| ---- | ---- | ---- | ---- |
|df1 > df2|	13.32|	125.35|	0.1063|
|df1 * df2|	21.71|	36.63|	0.5928|
|df1 + df2|	22.04|	36.50|	0.6039|


这两个支持库默认为启用状态，可用以下选项设置：

In [15]:
# pd.set_option('compute.use_bottleneck', False)
# pd.set_option('compute.use_numexpr', False)

## 二进制操作

Pandas 数据结构之间执行二进制操作，要注意下列两个关键点：
- 多维（DataFrame）与低维（Series）对象之间的广播机制；
- 计算中的缺失值处理。

这两个问题可以同时处理，但下面先介绍怎么分开处理。

### 广播机制


DataFrame 支持 add()、sub()、mul()、div()及 radd()、rsub()等方法执行二进制操作。广播机制重点关注输入的 Series。通过 axis 关键字，匹配 index 或 columns 即可调用这些函数。

In [16]:
df = pd.DataFrame({
    'one': pd.Series(np.random.randn(3), index=['a', 'b', 'c']),
    'two': pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']),
    'three': pd.Series(np.random.randn(3), index=['b', 'c', 'd'])})
df

,one,two,three
a,-1.196879,0.578051,NaN
b,-1.054653,-0.474366,0.191145
c,1.100068,-2.255187,1.916800
d,NaN,1.087245,-0.480380


In [17]:
row = df.iloc[1]
row

one     -1.054653
two     -0.474366
three    0.191145
Name: b, dtype: float64

In [18]:
column = df['two']
column

a    0.578051
b   -0.474366
c   -2.255187
d    1.087245
Name: two, dtype: float64

In [19]:
df.sub(row, axis='columns')

,one,two,three
a,-0.142225,1.052418,NaN
b,0.000000,0.000000,0.000000
c,2.154722,-1.780821,1.725655
d,NaN,1.561611,-0.671526


In [20]:
df.sub(row, axis=1)

,one,two,three
a,-0.142225,1.052418,NaN
b,0.000000,0.000000,0.000000
c,2.154722,-1.780821,1.725655
d,NaN,1.561611,-0.671526


In [21]:
df.sub(column, axis='index')

,one,two,three
a,-1.774930,0.0,NaN
b,-0.580287,0.0,0.665512
c,3.355255,0.0,4.171987
d,NaN,0.0,-1.567625


In [22]:
df.sub(column, axis=0)

,one,two,three
a,-1.774930,0.0,NaN
b,-0.580287,0.0,0.665512
c,3.355255,0.0,4.171987
d,NaN,0.0,-1.567625


还可以用 Series 对齐多层索引 DataFrame 的某一层级。

In [23]:
dfmi = df.copy()
dfmi

,one,two,three
a,-1.196879,0.578051,NaN
b,-1.054653,-0.474366,0.191145
c,1.100068,-2.255187,1.916800
d,NaN,1.087245,-0.480380


In [24]:
dfmi.index = pd.MultiIndex.from_tuples([(1, 'a'), (1, 'b'),
                                        (1, 'c'), (2, 'a')],
                                       names=['first', 'second'])
dfmi

one       two     three
first second                              
1     a      -1.196879  0.578051       NaN
      b      -1.054653 -0.474366  0.191145
      c       1.100068 -2.255187  1.916800
2     a            NaN  1.087245 -0.480380

In [25]:
dfmi.sub(column, axis=0, level='second')

one       two     three
first second                              
1     a      -1.774930  0.000000       NaN
      b      -0.580287  0.000000  0.665512
      c       3.355255  0.000000  4.171987
2     a            NaN  0.509194 -1.058432

Series 与 Index 还支持 divmod()内置函数，该函数同时执行向下取整除与模运算，返回两个与左侧类型相同的元组。示例如下：

In [26]:
s = pd.Series(np.arange(10))
s

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
dtype: int64

In [27]:
div, rem = divmod(s, 3)
div

0    0
1    0
2    0
3    1
4    1
5    1
6    2
7    2
8    2
9    3
dtype: int64

In [28]:
rem

0    0
1    1
2    2
3    0
4    1
5    2
6    0
7    1
8    2
9    0
dtype: int64

In [29]:
idx = pd.Index(np.arange(10))
idx

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')

### 缺失值与填充缺失值操作

Series 与 DataFrame 的算数函数支持 fill_value 选项，即用指定值替换某个位置的缺失值。比如，两个 DataFrame 相加，除非两个 DataFrame 里同一个位置都有缺失值，其相加的和仍为 NaN，如果只有一个 DataFrame 里存在缺失值，则可以用 fill_value 指定一个值来替代 NaN，当然，也可以用 fillna 把 NaN 替换为想要的值。

In [30]:
df2 = pd.DataFrame({'one': pd.Series(np.random.randn(3), index=['a', 'b', 'c']), 
                    'two': pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']), 
                    'three': pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd'])})
df2

,one,two,three
a,-1.179263,0.533547,1.143510
b,0.748569,0.377998,0.086031
c,-1.032834,0.202369,0.793054
d,NaN,-0.769855,0.015839


In [31]:
df

,one,two,three
a,-1.196879,0.578051,NaN
b,-1.054653,-0.474366,0.191145
c,1.100068,-2.255187,1.916800
d,NaN,1.087245,-0.480380


In [32]:
df + df2

,one,two,three
a,-2.376141,1.111598,NaN
b,-0.306085,-0.096368,0.277177
c,0.067234,-2.052818,2.709854
d,NaN,0.317390,-0.464541


In [33]:
df.add(df2, fill_value=0)

,one,two,three
a,-2.376141,1.111598,1.143510
b,-0.306085,-0.096368,0.277177
c,0.067234,-2.052818,2.709854
d,NaN,0.317390,-0.464541


### 比较操作

与上一小节的算数运算类似，Series 与 DataFrame 还支持 eq、ne、lt、gt、le、ge 等二进制比较操作的方法：  

|序号|缩写|英文|中文|
|----|----|:----|----|
|1|eq|equal to|等于|
|2|ne|notequal to|不等于|
|3|lt|less than|小于|
|4|gt|greater than|大于|
|5|le|less than or equal to	|小于等于|
|6|ge|greater than or equal to	|大于等于|


In [34]:
df.gt(df2)

,one,two,three
a,False,True,False
b,False,False,True
c,True,False,True
d,False,True,False


In [35]:
df2.ne(df)

,one,two,three
a,True,True,True
b,True,True,True
c,True,True,True
d,True,True,True


这些操作生成一个与左侧输入对象类型相同的 Pandas 对象，即，dtype 为 bool。boolean 对象可用于索引操作，参阅布尔索引。

### 布尔简化

empty、any()、all()、bool()可以把数据汇总简化至单个布尔值

In [36]:
(df > 0).all()

one      False
two      False
three    False
dtype: bool

In [37]:
(df > 0).any()

one      True
two      True
three    True
dtype: bool

In [38]:
# 还可以进一步把上面的结果简化为单个布尔值。
(df > 0).any().any()


True

通过empty属性，可以验证 Pandas 对象是否为空。

In [39]:
df.empty

False

In [40]:
pd.DataFrame(columns=list('ABC')).empty

True

用bool()方法验证单元素 pandas 对象的布尔值。

In [41]:
pd.Series([True]).bool()

True

In [42]:
pd.Series([False]).bool()

False

In [43]:
pd.DataFrame([[True]]).bool()

True

In [44]:
pd.DataFrame([[False]]).bool()

False

### 比较对象是否相等

一般情况下，多种方式都能得出相同的结果。以 df + df 与 df * 2 为例。应用上一小节学到的知识，测试这两种计算方式的结果是否一致，一般人都会用 (df + df == df * 2).all()，不过，这个表达式的结果是 False。

In [45]:
df + df == df * 2

,one,two,three
a,True,True,False
b,True,True,True
c,True,True,True
d,False,True,True


In [46]:
(df + df == df * 2).all()

one      False
two       True
three    False
dtype: bool

注意：布尔型 DataFrame df + df == df * 2 中有 False 值！这是因为两个 NaN 值的比较结果为不等：

In [47]:
np.nan == np.nan

False

为了验证数据是否等效，Series 与 DataFrame 等 N 维框架提供了 equals()方法，用这个方法验证 NaN 值的结果为相等。

In [48]:
(df + df).equals(df * 2)

True

注意：Series 与 DataFrame 索引的顺序必须一致，验证结果才能为 True

In [49]:
df1 = pd.DataFrame({'col': ['foo', 0, np.nan]})
df1

,col
0,foo
1,0
2,NaN


In [50]:
df2 = pd.DataFrame({'col': [np.nan, 0, 'foo']}, index=[2, 1, 0])
df2

,col
2,NaN
1,0
0,foo


In [51]:
df1.equals(df2)

False

In [52]:
df1.equals(df2.sort_index())

True

### 比较array型对象

用标量值与 Pandas 数据结构对比数据元素非常简单：

In [53]:
pd.Series(['foo', 'bar', 'baz']) == 'foo'

0     True
1    False
2    False
dtype: bool

In [54]:
pd.Index(['foo', 'bar', 'baz']) == 'foo'

array([ True, False, False])

Pandas 还能对比两个等长 array 对象里的数据元素：

In [55]:
pd.Series(['foo', 'bar', 'baz']) == pd.Index(['foo', 'bar', 'qux'])

0     True
1     True
2    False
dtype: bool

In [56]:
pd.Series(['foo', 'bar', 'baz']) == np.array(['foo', 'bar', 'qux'])

0     True
1     True
2    False
dtype: bool

对比不等长的 Index 或 Series 对象会触发 ValueError：

In [57]:
# pd.Series(['foo', 'bar', 'baz']) == pd.Series(['foo', 'bar'])

In [58]:
# pd.Series(['foo', 'bar', 'baz']) == pd.Series(['foo'])

注意： 这里的操作与 NumPy 的广播机制不同：

In [59]:
np.array([1, 2, 3]) == np.array([2])

array([False,  True, False])

NumPy 无法执行广播操作时，返回 False:

In [60]:
np.array([1, 2, 3]) == np.array([1, 3])

<ipython-input-60-bdfbc1819bb5>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  np.array([1, 2, 3]) == np.array([1, 3])


False

### 合并重叠数据集

有时，要合并两个相似的数据集，两个数据集里的其中一个的数据比另一个多。比如，展示特定经济指标的两个数据序列，其中一个是“高质量”指标，另一个是“低质量”指标。一般来说，低质量序列可能包含更多的历史数据，或覆盖更广的数据。因此，要合并这两个 DataFrame 对象，其中一个 DataFrame 中的缺失值将按指定条件用另一个 DataFrame 里类似标签中的数据进行填充。要实现这一操作，请用下列代码中的 combine_first()函数。

In [61]:
df1 = pd.DataFrame({'A': [1., np.nan, 3., 5., np.nan],
                    'B': [np.nan, 2., 3., np.nan, 6.]})
df1

,A,B
0,1.0,NaN
1,NaN,2.0
2,3.0,3.0
3,5.0,NaN
4,NaN,6.0


In [62]:
df2 = pd.DataFrame({'A': [5., 2., 4., np.nan, 3., 7.],
                    'B': [np.nan, np.nan, 3., 4., 6., 8.]})
df2

,A,B
0,5.0,NaN
1,2.0,NaN
2,4.0,3.0
3,NaN,4.0
4,3.0,6.0
5,7.0,8.0


In [63]:
df1.combine_first(df2)

,A,B
0,1.0,NaN
1,2.0,2.0
2,3.0,3.0
3,5.0,4.0
4,3.0,6.0
5,7.0,8.0


### DataFrame 通用合并方法

上述 combine_first方法调用了更普适的 DataFrame.combine()方法。该方法提取另一个 DataFrame 及合并器函数，并将之与输入的 DataFrame 对齐，再传递与 Series 配对的合并器函数（比如，名称相同的列）。  
下面的代码复现了上述的 combine_first()函数：

In [64]:
def combiner(x, y):
    return np.where(pd.isna(x), y, x)  # y if isna(x) else x   如果x为空就填入y的值，否则就填入x的值

## 描述性统计

Series与 DataFrame支持大量计算描述性统计的方法与操作。这些方法大部分都是 sum()、mean()、quantile()等聚合函数，其输出结果比原始数据集小；此外，还有输出结果与原始数据集同样大小的 cumsum()、 cumprod()等函数。这些方法都基本上都接受 axis 参数，如， ndarray.{sum,std,…}，但这里的 axis 可以用名称或整数指定：

Series：无需 axis 参数
DataFrame：
index，即 axis=0，默认值
columns, 即 axis=1

In [65]:
df

,one,two,three
a,-1.196879,0.578051,NaN
b,-1.054653,-0.474366,0.191145
c,1.100068,-2.255187,1.916800
d,NaN,1.087245,-0.480380


In [66]:
df.mean(0)

one     -0.383821
two     -0.266064
three    0.542522
dtype: float64

In [67]:
df.mean(1)

a   -0.309414
b   -0.445958
c    0.253894
d    0.303432
dtype: float64

上述方法都支持 skipna 关键字，指定是否要排除缺失数据，默认值为 True

In [68]:
df.sum(0, skipna=False)

one           NaN
two     -1.064257
three         NaN
dtype: float64

In [69]:
df.sum(axis=1, skipna=True)

a   -0.618827
b   -1.337874
c    0.761682
d    0.606865
dtype: float64

结合广播机制或算数操作，可以描述不同统计过程，比如标准化，即渲染数据零均值与标准差  
1，这种操作非常简单：


In [70]:
ts_stand = (df - df.mean()) / df.std()
ts_stand

,one,two,three
a,-0.631721,0.571534,NaN
b,-0.521216,-0.141037,-0.284144
c,1.152937,-1.346795,1.111322
d,NaN,0.916298,-0.827179


In [71]:
ts_stand.std()

one      1.0
two      1.0
three    1.0
dtype: float64

In [72]:
xs_stand = df.sub(df.mean(1), axis=0).div(df.std(1), axis=0)
xs_stand.std(1)

a    1.0
b    1.0
c    1.0
d    1.0
dtype: float64

注 ： cumsum()与 cumprod()等方法保留 NaN 值的位置。这与 expanding()和 rolling()略显不同。

In [73]:
df.cumsum()

,one,two,three
a,-1.196879,0.578051,NaN
b,-2.251532,0.103685,0.191145
c,-1.151463,-2.151502,2.107946
d,NaN,-1.064257,1.627566


下表为常用函数汇总表。每个函数都支持 level 参数，仅在数据对象为结构化 Index时使用。  

|函数|描述|
|:----|:----|
|count|统计非空值数量|
|sum|汇总值|
|mean|平均值|
|mad|平均绝对偏差|
|median|算数中位数|
|min|	最小值|
|max|	最大值|
|mode|	众数|
|abs|	绝对值|
|prod|	乘积|
|std|	贝塞尔校正的样本标准偏差|
|var|	无偏方差|
|sem|	平均值的标准误差|
|skew|	样本偏度 (第三阶)|
|kurt|	样本峰度 (第四阶)|
|quantile|	样本分位数 (不同 % 的值)|
|cumsum|	累加|
|cumprod|	累乘|
|cummax|	累积最大值|
|cummin|	累积最小值|

注意：NumPy 的 mean、std、sum 等方法默认不统计 Series 里的空值。

In [74]:
np.mean(df['one'])

-0.3838211615581955

In [75]:
np.mean(df['one'].to_numpy())

nan

Series.nunique()返回 Series 里所有非空值的唯一值。

In [76]:
series = pd.Series(np.random.randn(500))

In [77]:
series[20:500] = np.nan

In [78]:
series[10:20] = 5
series

0      0.013486
1      0.057805
2      1.195661
3      1.909973
4      2.846423
         ...   
495         NaN
496         NaN
497         NaN
498         NaN
499         NaN
Length: 500, dtype: float64

In [79]:
series.nunique()

11

### 数据总结：describe

describe()函数计算 Series 与 DataFrame 数据列的各种数据统计量，注意，这里排除了空值。

In [80]:
series = pd.Series(np.random.randn(1000))

In [81]:
series[::2] = np.nan

In [82]:
series.describe()

count    500.000000
mean       0.036206
std        1.011403
min       -3.793757
25%       -0.538167
50%        0.120324
75%        0.673197
max        2.976569
dtype: float64

In [83]:
frame = pd.DataFrame(np.random.randn(1000, 5),
                     columns=['a', 'b', 'c', 'd', 'e'])

In [84]:
frame.iloc[::2] = np.nan

In [85]:
frame.describe()

,a,b,c,d,e
count,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.041860,0.062873,-0.021412,0.103296,-0.033445
std,0.999257,1.026034,1.008423,1.012092,0.978852
min,-3.255237,-3.026724,-2.759187,-3.821282,-2.395802
25%,-0.747009,-0.679808,-0.702055,-0.562262,-0.755275
50%,0.083891,0.086254,-0.067729,0.078075,-0.080388
75%,0.795492,0.750347,0.682866,0.756694,0.586257
max,3.009059,3.918496,3.069452,3.114917,2.893645


此外，还可以指定输出结果包含的分位数：

In [86]:
series.describe(percentiles=[.05, .25, .75, .95])

count    500.000000
mean       0.036206
std        1.011403
min       -3.793757
5%        -1.746599
25%       -0.538167
50%        0.120324
75%        0.673197
95%        1.586131
max        2.976569
dtype: float64

cut() 函数（以值为依据实现分箱）及 qcut() 函数（以样本分位数为依据实现分箱）用于连续值的离散化：

In [87]:
arr = np.random.randn(20)
arr

array([-8.72384409e-01, -1.63389340e-01,  5.41324287e-01, -6.42880698e-01,
        1.22840798e-01,  7.87831710e-01, -6.62604307e-01, -1.63615580e+00,
       -1.42138754e+00,  5.86330748e-04, -2.77188309e+00, -2.30169395e-01,
       -7.38558524e-01, -8.21615665e-01,  4.59825914e-01, -1.60363153e-01,
       -5.31691183e-01, -2.44929427e+00, -5.81558930e-01,  1.20723220e+00])

In [88]:
factor = pd.cut(arr, 4)
factor

[(-1.777, -0.782], (-0.782, 0.212], (0.212, 1.207], (-0.782, 0.212], (-0.782, 0.212], ..., (-0.782, 0.212], (-0.782, 0.212], (-2.776, -1.777], (-0.782, 0.212], (0.212, 1.207]]
Length: 20
Categories (4, interval[float64]): [(-2.776, -1.777] < (-1.777, -0.782] < (-0.782, 0.212] < (0.212, 1.207]]

In [89]:
factor = pd.cut(arr, [-5, -1, 0, 1, 5])
factor

[(-1, 0], (-1, 0], (0, 1], (-1, 0], (0, 1], ..., (-1, 0], (-1, 0], (-5, -1], (-1, 0], (1, 5]]
Length: 20
Categories (4, interval[int64]): [(-5, -1] < (-1, 0] < (0, 1] < (1, 5]]

qcut()计算样本分位数。比如，下列代码按等距分位数分割正态分布的数据：

In [90]:
arr = np.random.randn(30)
arr

array([-4.01388883e-01, -4.90146666e-01,  7.25117639e-01,  6.91570892e-01,
       -7.48929846e-01, -6.00668266e-01,  5.53152785e-01,  1.36707119e-01,
        1.01976824e-03, -7.43035807e-02, -2.75100987e-01,  1.38063907e+00,
        2.42084658e-02, -1.99933586e+00, -3.04505196e-01, -1.77752579e+00,
       -1.10456374e+00,  9.14705675e-01,  5.87430699e-01,  8.84769495e-01,
        9.21972934e-01, -2.15144933e+00,  1.70320126e-02,  2.31644699e-01,
       -6.62931792e-01, -1.09612060e-01,  7.15106345e-01, -1.17947295e+00,
       -1.13384560e+00, -1.61233294e-01])

In [91]:
factor = pd.qcut(arr, [0, .25, .5, .75, 1])
factor

[(-0.647, -0.092], (-0.647, -0.092], (0.579, 1.381], (0.579, 1.381], (-2.1519999999999997, -0.647], ..., (-0.647, -0.092], (0.579, 1.381], (-2.1519999999999997, -0.647], (-2.1519999999999997, -0.647], (-0.647, -0.092]]
Length: 30
Categories (4, interval[float64]): [(-2.1519999999999997, -0.647] < (-0.647, -0.092] < (-0.092, 0.579] < (0.579, 1.381]]

In [92]:
pd.value_counts(factor)

(-2.1519999999999997, -0.647]    8
(0.579, 1.381]                   8
(-0.647, -0.092]                 7
(-0.092, 0.579]                  7
dtype: int64

In [93]:
arr = np.random.randn(20)

In [94]:
factor = pd.cut(arr, [-np.inf, 0, np.inf])
factor

[(0.0, inf], (-inf, 0.0], (-inf, 0.0], (-inf, 0.0], (0.0, inf], ..., (-inf, 0.0], (0.0, inf], (0.0, inf], (-inf, 0.0], (0.0, inf]]
Length: 20
Categories (2, interval[float64]): [(-inf, 0.0] < (0.0, inf]]

## 函数应用

不管是为 Pandas 对象应用自定义函数，还是应用第三方函数，都离不开以下三种方法。用哪种方法取决于操作的对象是 DataFrame，还是 Series ；是行、列，还是元素。
1. 表级函数应用：pipe()
2. 行列级函数应用： apply()
3. 聚合 API： agg() 与 transform()
4. 元素级函数应用：applymap()

### 表级函数应用

虽然可以把 DataFrame 与 Series 传递给函数，不过链式调用函数时，最好使用 pipe()方法。对比以下两种方式：

In [95]:
# f、g、h 是提取、返回 `DataFrames` 的函数
# f(g(h(df), arg1=1), arg2=2, arg3=3)

In [96]:
# (df.pipe(h).pipe(g, arg1=1).pipe(f, arg2=2, arg3=3)

Pandas 鼓励使用第二种方式，即链式方法。在链式方法中调用自定义函数或第三方支持库函数时，用 pipe 更容易，与用 Pandas 自身方法一样。

上例中，f、g 与 h 这几个函数都把 DataFrame 当作首位参数。要是想把数据作为第二个参数，该怎么办？本例中，pipe 为元组 （callable,data_keyword）形式。.pipe 把 DataFrame 作为元组里指定的参数。

下例用 statsmodels 拟合回归。该 API 先接收一个公式，DataFrame 是第二个参数，data。要传递函数，则要用pipe 接收关键词对 (sm.ols,'data')。

In [97]:
import statsmodels.formula.api as sm

In [98]:
bb = pd.read_csv('../data/baseball.csv', index_col='id')
bb

,player,year,stint,team,lg,g,ab,r,h,X2b,...,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp
id,,,,,,,,,,,,,,,,,,,,,
88641,womacto01,2006,2,CHN,NL,19,50,6,14,1,...,2.0,1.0,1.0,4,4.0,0.0,0.0,3.0,0.0,0.0
88643,schilcu01,2006,1,BOS,AL,31,2,0,1,0,...,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0
88645,myersmi01,2006,1,NYA,AL,62,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
88649,helliri01,2006,1,MIL,NL,20,3,0,0,0,...,0.0,0.0,0.0,0,2.0,0.0,0.0,0.0,0.0,0.0
88650,johnsra05,2006,1,NYA,AL,33,6,0,1,0,...,0.0,0.0,0.0,0,4.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89525,benitar01,2007,2,FLO,NL,34,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
89526,benitar01,2007,1,SFN,NL,19,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
89530,ausmubr01,2007,1,HOU,NL,117,349,38,82,16,...,25.0,6.0,1.0,37,74.0,3.0,6.0,4.0,1.0,11.0


In [99]:
(bb.query('h > 0')
 .assign(ln_h=lambda df: np.log(df.h))
 .pipe((sm.ols, 'data'), 'hr ~ ln_h + year + g + C(lg)')
 .fit()
 .summary()
)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     hr   R-squared:                       0.685
Model:                            OLS   Adj. R-squared:                  0.665
Method:                 Least Squares   F-statistic:                     34.28
Date:                Wed, 14 Apr 2021   Prob (F-statistic):           3.48e-15
Time:                        09:57:51   Log-Likelihood:                -205.92
No. Observations:                  68   AIC:                             421.8
Df Residuals:                      63   BIC:                             432.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -8484.7720   4664.146     -1.819      0.074   -1.78e+04     835.780
C(lg)[T.NL]    -2.2736      1.325     -1.716      0.091      -4.922       0.375
ln_h           -1.3542      0.875     -1.547      0.127      -3.103       0.395
year            4.2277      2.324      1.819      0.074      -0.417       8.872
g               0.1841      0.029      6.258      0.000       0.125       0.243
==============================================================================
Omnibus:                       10.875   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               17.298
Skew:                           0.537   Prob(JB):                     0.000175
Kurtosis:                       5.225   Cond. No.                     1.49e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.49e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

unix 的 pipe 与后来出现的 dplyr (opens new window)及 magrittr (opens new window)启发了pipe 方法，在此，引入了 R 语言里用于读取 pipe 的操作符 (%>%)。pipe 的实现思路非常清晰，仿佛 Python 源生的一样。强烈建议大家阅读 pipe() (opens new window)的源代码。

### 行列级函数应用

apply()方法沿着 DataFrame 的轴应用函数，比如，描述性统计方法，该方法支持 axis 参数

In [100]:
df.apply(np.mean)

one     -0.383821
two     -0.266064
three    0.542522
dtype: float64

In [101]:
df.apply(np.mean, axis=1)

a   -0.309414
b   -0.445958
c    0.253894
d    0.303432
dtype: float64

In [102]:
df.apply(lambda x: x.max() - x.min())

one      2.296947
two      3.342432
three    2.397181
dtype: float64

In [103]:
df.apply(np.cumsum)

,one,two,three
a,-1.196879,0.578051,NaN
b,-2.251532,0.103685,0.191145
c,-1.151463,-2.151502,2.107946
d,NaN,-1.064257,1.627566


In [104]:
df.apply(np.exp)

,one,two,three
a,0.302136,1.782562,NaN
b,0.348313,0.622279,1.210636
c,3.004371,0.104854,6.799169
d,NaN,2.966091,0.618548


apply()方法还支持通过函数名字符串调用函数

In [105]:
df.apply('mean')

one     -0.383821
two     -0.266064
three    0.542522
dtype: float64

In [106]:
df.apply('mean', axis=1)

a   -0.309414
b   -0.445958
c    0.253894
d    0.303432
dtype: float64

默认情况下，apply()调用的函数返回的类型会影响 DataFrame.apply 输出结果的类型。
- 函数返回的是 Series 时，最终输出结果是 DataFrame。输出的列与函数返回的 Series 索引相匹配。
- 函数返回其它任意类型时，输出结果是 Series。 

result_type 会覆盖默认行为，该参数有三个选项：reduce、broadcast、expand。这些选项决定了列表型返回值是否扩展为 DataFrame。

In [107]:
tsdf = pd.DataFrame(np.random.randn(1000, 3), columns=['A', 'B', 'C'],
                    index=pd.date_range('1/1/2000', periods=1000))
tsdf.apply(lambda x: x.idxmax())

A   2002-06-24
B   2001-04-15
C   2002-08-01
dtype: datetime64[ns]

还可以向 apply()方法传递额外的参数与关键字参数。比如下例中要应用的这个函数：

In [108]:
def subtract_and_divide(x, sub, divide=1):
    return (x - sub) / divide

In [109]:
df4 = df.apply(subtract_and_divide, args=(5,), divide=3)
df4

,one,two,three
a,-2.065626,-1.473983,NaN
b,-2.018218,-1.824789,-1.602952
c,-1.299977,-2.418396,-1.027733
d,NaN,-1.304252,-1.826793


In [110]:
tsdf

,A,B,C
2000-01-01,-0.598639,1.001630,-2.113701
2000-01-02,-0.178734,-1.071798,0.394691
2000-01-03,-0.256316,0.847235,0.618951
2000-01-04,1.202768,0.640080,-0.019449
2000-01-05,-2.622882,0.144324,-0.271216
...,...,...,...
2002-09-22,-0.145237,-1.762433,-1.412179
2002-09-23,0.619972,-1.202077,0.045095
2002-09-24,0.795336,-0.204636,-0.529487
2002-09-25,0.677615,-0.171197,-0.468369


In [111]:
tsdf.apply(pd.Series.interpolate)

,A,B,C
2000-01-01,-0.598639,1.001630,-2.113701
2000-01-02,-0.178734,-1.071798,0.394691
2000-01-03,-0.256316,0.847235,0.618951
2000-01-04,1.202768,0.640080,-0.019449
2000-01-05,-2.622882,0.144324,-0.271216
...,...,...,...
2002-09-22,-0.145237,-1.762433,-1.412179
2002-09-23,0.619972,-1.202077,0.045095
2002-09-24,0.795336,-0.204636,-0.529487
2002-09-25,0.677615,-0.171197,-0.468369


apply()有一个参数 raw，默认值为 False，在应用函数前，使用该参数可以将每行或列转换为 Series。该参数为 True 时，传递的函数接收 ndarray 对象，若不需要索引功能，这种操作能显著提高性能。

### 聚合

聚合 API 可以快速、简洁地执行多个聚合操作。Pandas 对象支持多个类似的 API，如 groupby API、window functions API、resample API。聚合函数为DataFrame.aggregate()，它的别名是 DataFrame.agg()。   

此处用与上例类似的 DataFrame：

In [112]:
tsdf = pd.DataFrame(np.random.randn(10, 3), columns=['A', 'B', 'C'],
                    index=pd.date_range('1/1/2000', periods=10))

In [113]:
tsdf.iloc[3:7] = np.nan

In [114]:
tsdf

,A,B,C
2000-01-01,-0.477905,-1.998633,-0.064480
2000-01-02,-0.927048,0.218353,1.816695
2000-01-03,-1.024215,0.310065,-1.165125
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,-0.263068,-1.202980,-0.272642
2000-01-09,0.626097,0.735332,-0.209977
2000-01-10,0.913915,-0.529856,0.423761


应用单个函数时，该操作与 apply()等效，这里也可以用字符串表示聚合函数名。下面的聚合函数输出的结果为 Series：

In [115]:
tsdf.agg(np.sum)

A   -1.152224
B   -2.467719
C    0.528232
dtype: float64

In [116]:
tsdf.agg('sum')

A   -1.152224
B   -2.467719
C    0.528232
dtype: float64

In [117]:
tsdf.sum()

A   -1.152224
B   -2.467719
C    0.528232
dtype: float64

Series 单个聚合操作返回标量值：

In [118]:
tsdf.A.agg('sum')

-1.1522235403469132

### 多函数聚合
还可以用列表形式传递多个聚合函数。每个函数在输出结果 DataFrame 里以行的形式显示，行名是每个聚合函数的函数名。

In [119]:
tsdf.agg(['sum'])

,A,B,C
sum,-1.152224,-2.467719,0.528232


多个函数输出多行：

In [120]:
tsdf.agg(['sum', 'mean'])

,A,B,C
sum,-1.152224,-2.467719,0.528232
mean,-0.192037,-0.411286,0.088039


Series 聚合多函数返回结果还是 Series，索引为函数名：

In [121]:
tsdf.A.agg(['sum', 'mean'])

sum    -1.152224
mean   -0.192037
Name: A, dtype: float64

传递 lambda 函数时，输出名为 <lambda> 的行：

In [122]:
tsdf.A.agg(['sum', lambda x: x.mean()])

sum        -1.152224
<lambda>   -0.192037
Name: A, dtype: float64

应用自定义函数时，该函数名为输出结果的行名：

In [123]:
def mymean(x):
    return x.mean()

In [124]:
tsdf.A.agg(['sum', mymean])

sum      -1.152224
mymean   -0.192037
Name: A, dtype: float64

### 用字典实现聚合
指定为哪些列应用哪些聚合函数时，需要把包含列名与标量（或标量列表）的字典传递给 DataFrame.agg。  
注意：这里输出结果的顺序不是固定的，要想让输出顺序与输入顺序一致，请使用 OrderedDict。

In [125]:
tsdf.agg({'A': 'mean', 'B': 'sum'})

A   -0.192037
B   -2.467719
dtype: float64

输入的参数是列表时，输出结果为 DataFrame，并以矩阵形式显示所有聚合函数的计算结果，且输出结果由所有唯一函数组成。未执行聚合操作的列输出结果为 NaN 值：

In [126]:
tsdf.agg({'A': ['mean', 'min'], 'B': 'sum'})

,A,B
mean,-0.192037,NaN
min,-1.024215,NaN
sum,NaN,-2.467719


### 多种数据类型（Dtype）

与 groupby 的 .agg 操作类似，DataFrame 含不能执行聚合的数据类型时，.agg 只计算可聚合的列：

In [127]:
mdf = pd.DataFrame({'A': [1, 2, 3],
                    'B': [1., 2., 3.],
                    'C': ['foo', 'bar', 'baz'],
                    'D': pd.date_range('20130101', periods=3)})

In [128]:
mdf.dtypes

A             int64
B           float64
C            object
D    datetime64[ns]
dtype: object

In [129]:
mdf.agg(['min', 'sum'])

,A,B,C,D
min,1,1.0,bar,2013-01-01
sum,6,6.0,foobarbaz,NaT


### 自定义 Describe

.agg() 可以创建类似于内置 describe 函数的自定义 describe 函数。

In [130]:
from functools import partial

In [131]:
q_25 = partial(pd.Series.quantile, q=0.25)
q_25.__name__ = '25%'

In [132]:
q_75 = partial(pd.Series.quantile, q=0.75)
q_75.__name__ = '75%'

In [133]:
tsdf.agg(['count', 'mean', 'std', 'min', q_25, 'median', q_75, 'max'])

,A,B,C
count,6.000000,6.000000,6.000000
mean,-0.192037,-0.411286,0.088039
std,0.801499,1.038464,0.991317
min,-1.024215,-1.998633,-1.165125
25%,-0.814762,-1.034699,-0.256976
median,-0.370486,-0.155751,-0.137229
75%,0.403806,0.287137,0.301701
max,0.913915,0.735332,1.816695


### Transform API

transform()方法的返回结果与原始数据的索引相同，大小相同。与 .agg API 类似，该 API 支持同时处理多种操作，不用一个一个操作。   
下面，先创建一个 DataFrame：

In [134]:
tsdf = pd.DataFrame(np.random.randn(10, 3), columns=['A', 'B', 'C'],
                    index=pd.date_range('1/1/2000', periods=10))

In [135]:
tsdf.iloc[3:7] = np.nan

In [136]:
tsdf

,A,B,C
2000-01-01,-0.501184,-0.633370,-0.466654
2000-01-02,-0.668935,0.383871,-1.852100
2000-01-03,-1.149682,-0.282233,-0.242465
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,0.265383,1.272769,0.321668
2000-01-09,0.829702,1.014427,0.421304
2000-01-10,2.527259,-0.706858,1.354275


这里转换的是整个 DataFrame。.transform() 支持 NumPy 函数、字符串函数及自定义函数。

In [137]:
tsdf.transform(np.abs)

,A,B,C
2000-01-01,0.501184,0.633370,0.466654
2000-01-02,0.668935,0.383871,1.852100
2000-01-03,1.149682,0.282233,0.242465
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,0.265383,1.272769,0.321668
2000-01-09,0.829702,1.014427,0.421304
2000-01-10,2.527259,0.706858,1.354275


In [138]:
tsdf.transform('abs')

,A,B,C
2000-01-01,0.501184,0.633370,0.466654
2000-01-02,0.668935,0.383871,1.852100
2000-01-03,1.149682,0.282233,0.242465
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,0.265383,1.272769,0.321668
2000-01-09,0.829702,1.014427,0.421304
2000-01-10,2.527259,0.706858,1.354275


In [139]:
tsdf.transform(lambda x: x.abs())

,A,B,C
2000-01-01,0.501184,0.633370,0.466654
2000-01-02,0.668935,0.383871,1.852100
2000-01-03,1.149682,0.282233,0.242465
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,0.265383,1.272769,0.321668
2000-01-09,0.829702,1.014427,0.421304
2000-01-10,2.527259,0.706858,1.354275


这里的 transform()接受单个函数；与 ufunc 等效。

In [140]:
np.abs(tsdf)

,A,B,C
2000-01-01,0.501184,0.633370,0.466654
2000-01-02,0.668935,0.383871,1.852100
2000-01-03,1.149682,0.282233,0.242465
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,0.265383,1.272769,0.321668
2000-01-09,0.829702,1.014427,0.421304
2000-01-10,2.527259,0.706858,1.354275


.transform() 向 Series 传递单个函数时，返回的结果也是单个 Series。

In [141]:
tsdf.A.transform(np.abs)

2000-01-01    0.501184
2000-01-02    0.668935
2000-01-03    1.149682
2000-01-04         NaN
2000-01-05         NaN
2000-01-06         NaN
2000-01-07         NaN
2000-01-08    0.265383
2000-01-09    0.829702
2000-01-10    2.527259
Freq: D, Name: A, dtype: float64

### 多函数 Transform
transform() 调用多个函数时，生成多层索引 DataFrame。第一层是原始数据集的列名；第二层是 transform() 调用的函数名。

In [142]:
tsdf.transform([np.abs, lambda x: x + 1])

A                   B                   C          
            absolute  <lambda>  absolute  <lambda>  absolute  <lambda>
2000-01-01  0.501184  0.498816  0.633370  0.366630  0.466654  0.533346
2000-01-02  0.668935  0.331065  0.383871  1.383871  1.852100 -0.852100
2000-01-03  1.149682 -0.149682  0.282233  0.717767  0.242465  0.757535
2000-01-04       NaN       NaN       NaN       NaN       NaN       NaN
2000-01-05       NaN       NaN       NaN       NaN       NaN       NaN
2000-01-06       NaN       NaN       NaN       NaN       NaN       NaN
2000-01-07       NaN       NaN       NaN       NaN       NaN       NaN
2000-01-08  0.265383  1.265383  1.272769  2.272769  0.321668  1.321668
2000-01-09  0.829702  1.829702  1.014427  2.014427  0.421304  1.421304
2000-01-10  2.527259  3.527259  0.706858  0.293142  1.354275  2.354275

为 Series 应用多个函数时，输出结果是 DataFrame，列名是 transform() 调用的函数名。

In [143]:
tsdf.A.transform([np.abs, lambda x: x + 1])

,absolute,<lambda>
2000-01-01,0.501184,0.498816
2000-01-02,0.668935,0.331065
2000-01-03,1.149682,-0.149682
2000-01-04,NaN,NaN
2000-01-05,NaN,NaN
2000-01-06,NaN,NaN
2000-01-07,NaN,NaN
2000-01-08,0.265383,1.265383
2000-01-09,0.829702,1.829702
2000-01-10,2.527259,3.527259


### 用字典执行 transform 操作
函数字典可以为每列执行指定 transform() 操作。

In [144]:
tsdf.transform({'A': np.abs, 'B': lambda x: x + 1})

,A,B
2000-01-01,0.501184,0.366630
2000-01-02,0.668935,1.383871
2000-01-03,1.149682,0.717767
2000-01-04,NaN,NaN
2000-01-05,NaN,NaN
2000-01-06,NaN,NaN
2000-01-07,NaN,NaN
2000-01-08,0.265383,2.272769
2000-01-09,0.829702,2.014427
2000-01-10,2.527259,0.293142


transform() 的参数是列表字典时，生成的是以 transform() 调用的函数为名的多层索引 DataFrame。

In [145]:
tsdf.transform({'A': np.abs, 'B': [lambda x: x + 1, 'sqrt']})

/Users/tianjun/.pyenv/versions/3.9.1/lib/python3.9/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


A         B          
                   A  <lambda>      sqrt
2000-01-01  0.501184  0.366630       NaN
2000-01-02  0.668935  1.383871  0.619574
2000-01-03  1.149682  0.717767       NaN
2000-01-04       NaN       NaN       NaN
2000-01-05       NaN       NaN       NaN
2000-01-06       NaN       NaN       NaN
2000-01-07       NaN       NaN       NaN
2000-01-08  0.265383  2.272769  1.128171
2000-01-09  0.829702  2.014427  1.007188
2000-01-10  2.527259  0.293142       NaN

### 元素级函数应用

并非所有函数都能矢量化，即接受 NumPy 数组，返回另一个数组或值，DataFrame 的 applymap()及 Series 的 map()，支持任何接收单个值并返回单个值的 Python 函数。

In [146]:
# df4 = pd.DataFrame(np.random.randn(4, 3), columns=['one', 'two', 'three'], index=['a', 'b', 'c', 'd'])
df4

,one,two,three
a,-2.065626,-1.473983,NaN
b,-2.018218,-1.824789,-1.602952
c,-1.299977,-2.418396,-1.027733
d,NaN,-1.304252,-1.826793


In [147]:
def f(x):
    return len(str(x))

In [148]:
df4['one'].map(f)

a    19
b    19
c    19
d     3
Name: one, dtype: int64

Series.map()还有个功能，可以“连接”或“映射”第二个 Series 定义的值。这与 merging / joining 功能联系非常紧密：

In [149]:
s = pd.Series(['six', 'seven', 'six', 'seven', 'six'],
              index=['a', 'b', 'c', 'd', 'e'])
s

a      six
b    seven
c      six
d    seven
e      six
dtype: object

In [150]:
t = pd.Series({'six': 6., 'seven': 7.})
t

six      6.0
seven    7.0
dtype: float64

In [151]:
s.map(t)

a    6.0
b    7.0
c    6.0
d    7.0
e    6.0
dtype: float64

## 重置索引与更换标签

reindex()是 Pandas 里实现数据对齐的基本方法，该方法执行几乎所有功能都要用到的标签对齐功能。 reindex 指的是沿着指定轴，让数据与给定的一组标签进行匹配。该功能完成以下几项操作：
- 让现有数据匹配一组新标签，并重新排序；
- 在无数据但有标签的位置插入缺失值（NA）标记；
- 如果指定，则按逻辑填充无标签的数据，该操作多见于时间序列数据。



In [152]:
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])
s

a   -0.242338
b   -0.292857
c    0.886394
d    0.895207
e    0.854163
dtype: float64

In [153]:
s.reindex(['e', 'b', 'f', 'd'])

e    0.854163
b   -0.292857
f         NaN
d    0.895207
dtype: float64

本例中，原 Series 里没有标签 f ，因此，输出结果里 f 对应的值为 NaN。   
DataFrame 支持同时 reindex 索引与列：

In [154]:
df.reindex(index=['c', 'f', 'b'], columns=['three', 'two', 'one'])

,three,two,one
c,1.916800,-2.255187,1.100068
f,NaN,NaN,NaN
b,0.191145,-0.474366,-1.054653


reindex 还支持 axis 关键字：

In [155]:
df.reindex(['c', 'f', 'b'], axis='index')

,one,two,three
c,1.100068,-2.255187,1.916800
f,NaN,NaN,NaN
b,-1.054653,-0.474366,0.191145


注意：不同对象可以共享 Index 包含的轴标签。比如，有一个 Series，还有一个 DataFrame，可以执行下列操作：

In [156]:
rs = s.reindex(df.index)
rs

a   -0.242338
b   -0.292857
c    0.886394
d    0.895207
dtype: float64

In [157]:
rs.index is df.index

True

这里指的是，重置后，Series 的索引与 DataFrame 的索引是同一个 Python 对象。

DataFrame.reindex()还支持 “轴样式”调用习语，可以指定单个 labels 参数，并指定应用于哪个 axis。

In [158]:
df.reindex(['c', 'f', 'b'], axis='index')

,one,two,three
c,1.100068,-2.255187,1.916800
f,NaN,NaN,NaN
b,-1.054653,-0.474366,0.191145


In [159]:
df.reindex(['three', 'two', 'one'], axis='columns')

,three,two,one
a,NaN,0.578051,-1.196879
b,0.191145,-0.474366,-1.054653
c,1.916800,-2.255187,1.100068
d,-0.480380,1.087245,NaN


> 编写注重性能的代码时，最好花些时间深入理解 reindex：预对齐数据后，操作会更快。两个未对齐的 DataFrame 相加，后台操作会执行 reindex。探索性分析时很难注意到这点有什么不同，这是因为 reindex 已经进行了高度优化，但需要注重 CPU 周期时，显式调用 reindex 还是有一些影响的。

### 重置索引，并与其它对象对齐 
提取一个对象，并用另一个具有相同标签的对象 reindex 该对象的轴。这种操作的语法虽然简单，但未免有些啰嗦。这时，最好用 reindex_like() 方法，这是一种既有效，又简单的方式：

In [160]:
df2 = pd.DataFrame(np.random.randn(3, 2), index=['a', 'b', 'c'], columns=['one', 'two'])
df2

,one,two
a,-0.924502,1.357980
b,-0.503811,1.512203
c,0.596151,-0.165982


In [161]:
df3 = pd.DataFrame(np.random.randn(3, 2), index=['a', 'b', 'c'], columns=['one', 'two'])
df3

,one,two
a,0.223047,-0.343141
b,-0.025812,-0.194289
c,1.686161,-0.180841


In [162]:
df.reindex_like(df2)

,one,two
a,-1.196879,0.578051
b,-1.054653,-0.474366
c,1.100068,-2.255187


In [163]:
df.reindex(df2.index)

,one,two,three
a,-1.196879,0.578051,NaN
b,-1.054653,-0.474366,0.191145
c,1.100068,-2.255187,1.916800


### 用 align 对齐多个对象
align()方法是对齐两个对象最快的方式，该方法支持 join 参数（请参阅 joining 与 merging）：
- `join='outer'`：使用两个对象索引的合集，默认值
- `join='left'`：使用左侧调用对象的索引
- `join='right'`：使用右侧传递对象的索引
- `join='inner'`：使用两个对象索引的交集

In [164]:
s = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])
s

a    0.234266
b   -0.852730
c    1.764455
d    0.340539
e   -0.359231
dtype: float64

In [165]:
s1 = s[:4]
s1

a    0.234266
b   -0.852730
c    1.764455
d    0.340539
dtype: float64

In [166]:
s2 = s[1:]

In [167]:
s1.align(s2)

(a    0.234266
 b   -0.852730
 c    1.764455
 d    0.340539
 e         NaN
 dtype: float64,
 a         NaN
 b   -0.852730
 c    1.764455
 d    0.340539
 e   -0.359231
 dtype: float64)

In [168]:
s1.align(s2, join='inner')

(b   -0.852730
 c    1.764455
 d    0.340539
 dtype: float64,
 b   -0.852730
 c    1.764455
 d    0.340539
 dtype: float64)

In [169]:
s1.align(s2, join='left')

(a    0.234266
 b   -0.852730
 c    1.764455
 d    0.340539
 dtype: float64,
 a         NaN
 b   -0.852730
 c    1.764455
 d    0.340539
 dtype: float64)

In [170]:
s1.align(s2, join='right')

(b   -0.852730
 c    1.764455
 d    0.340539
 e         NaN
 dtype: float64,
 b   -0.852730
 c    1.764455
 d    0.340539
 e   -0.359231
 dtype: float64)

默认条件下， join 方法既应用于索引，也应用于列：

In [171]:
df.align(df2, join='inner')

(        one       two
 a -1.196879  0.578051
 b -1.054653 -0.474366
 c  1.100068 -2.255187,
         one       two
 a -0.924502  1.357980
 b -0.503811  1.512203
 c  0.596151 -0.165982)

align 方法还支持 axis 选项，用来指定要对齐的轴：

In [172]:
df.align(df2, join='inner', axis=0)

(        one       two     three
 a -1.196879  0.578051       NaN
 b -1.054653 -0.474366  0.191145
 c  1.100068 -2.255187  1.916800,
         one       two
 a -0.924502  1.357980
 b -0.503811  1.512203
 c  0.596151 -0.165982)

如果把 Series 传递给 DataFrame.align() (opens new window)，可以用 axis 参数选择是在 DataFrame 的索引，还是列上对齐两个对象：

In [173]:
df.align(df2.iloc[0], axis=1)

(        one     three       two
 a -1.196879       NaN  0.578051
 b -1.054653  0.191145 -0.474366
 c  1.100068  1.916800 -2.255187
 d       NaN -0.480380  1.087245,
 one     -0.924502
 three         NaN
 two      1.357980
 Name: a, dtype: float64)

|方法|动作|
|:----|:----|
|pad / ffill|先前填充|
|bfill / backfill|向后填充|
|nearest|从最近的索引值填充|

下面用一个简单的 Series 展示 fill 方法：

In [174]:
rng = pd.date_range('1/3/2000', periods=8)
rng

DatetimeIndex(['2000-01-03', '2000-01-04', '2000-01-05', '2000-01-06',
               '2000-01-07', '2000-01-08', '2000-01-09', '2000-01-10'],
              dtype='datetime64[ns]', freq='D')

In [175]:
ts = pd.Series(np.random.randn(8), index=rng)
ts

2000-01-03    0.141469
2000-01-04    0.267447
2000-01-05    0.386353
2000-01-06    0.380289
2000-01-07   -0.144605
2000-01-08   -0.294344
2000-01-09   -1.353415
2000-01-10   -0.799587
Freq: D, dtype: float64

In [176]:
ts2 = ts[[0, 3, 6]]
ts2

2000-01-03    0.141469
2000-01-06    0.380289
2000-01-09   -1.353415
Freq: 3D, dtype: float64

In [177]:
ts2.reindex(ts.index)

2000-01-03    0.141469
2000-01-04         NaN
2000-01-05         NaN
2000-01-06    0.380289
2000-01-07         NaN
2000-01-08         NaN
2000-01-09   -1.353415
2000-01-10         NaN
Freq: D, dtype: float64

In [178]:
ts2.reindex(ts.index, method='ffill')

2000-01-03    0.141469
2000-01-04    0.141469
2000-01-05    0.141469
2000-01-06    0.380289
2000-01-07    0.380289
2000-01-08    0.380289
2000-01-09   -1.353415
2000-01-10   -1.353415
Freq: D, dtype: float64

In [179]:
ts2.reindex(ts.index, method='bfill')

2000-01-03    0.141469
2000-01-04    0.380289
2000-01-05    0.380289
2000-01-06    0.380289
2000-01-07   -1.353415
2000-01-08   -1.353415
2000-01-09   -1.353415
2000-01-10         NaN
Freq: D, dtype: float64

In [180]:
ts2.reindex(ts.index, method='nearest')

2000-01-03    0.141469
2000-01-04    0.141469
2000-01-05    0.380289
2000-01-06    0.380289
2000-01-07    0.380289
2000-01-08   -1.353415
2000-01-09   -1.353415
2000-01-10   -1.353415
Freq: D, dtype: float64

上述操作要求索引按递增或递减排序。  
注意：除了 method='nearest'，用fillna或interpolate也能实现同样的效果：

In [181]:
ts2.reindex(ts.index).fillna(method='ffill')

2000-01-03    0.141469
2000-01-04    0.141469
2000-01-05    0.141469
2000-01-06    0.380289
2000-01-07    0.380289
2000-01-08    0.380289
2000-01-09   -1.353415
2000-01-10   -1.353415
Freq: D, dtype: float64

如果索引不是按递增或递减排序，reindex()会触发 ValueError 错误。fillna()与 interpolate()则不检查索引的排序。

### 重置索引填充的限制
limit 与 tolerance 参数可以控制 reindex 的填充操作。limit 限定了连续匹配的最大数量：

In [182]:
ts2.reindex(ts.index, method='ffill', limit=1)

2000-01-03    0.141469
2000-01-04    0.141469
2000-01-05         NaN
2000-01-06    0.380289
2000-01-07    0.380289
2000-01-08         NaN
2000-01-09   -1.353415
2000-01-10   -1.353415
Freq: D, dtype: float64

反之，tolerance 限定了索引与索引器值之间的最大距离：

In [183]:
ts2.reindex(ts.index, method='ffill', tolerance='1 day')

2000-01-03    0.141469
2000-01-04    0.141469
2000-01-05         NaN
2000-01-06    0.380289
2000-01-07    0.380289
2000-01-08         NaN
2000-01-09   -1.353415
2000-01-10   -1.353415
Freq: D, dtype: float64

注意：索引为 DatetimeIndex、TimedeltaIndex 或 PeriodIndex 时，tolerance 会尽可能将这些索引强制转换为 Timedelta，这里要求用户用恰当的字符串设定 tolerance 参数。

### 去掉轴上的标签
drop()函数与 reindex 经常配合使用，该函数用于删除轴上的一组标签：

In [184]:
df

,one,two,three
a,-1.196879,0.578051,NaN
b,-1.054653,-0.474366,0.191145
c,1.100068,-2.255187,1.916800
d,NaN,1.087245,-0.480380


In [185]:
df.drop(['a', 'b'], axis=0)

,one,two,three
c,1.100068,-2.255187,1.91680
d,NaN,1.087245,-0.48038


In [186]:
df.drop(['one'], axis=1)

,two,three
a,0.578051,NaN
b,-0.474366,0.191145
c,-2.255187,1.916800
d,1.087245,-0.480380


注意：下面的代码可以运行，但不够清晰：

In [187]:
df.reindex(df.index.difference(['a', 'd']))

,one,two,three
b,-1.054653,-0.474366,0.191145
c,1.100068,-2.255187,1.916800


### 重命名或映射标签
rename()方法支持按不同的轴基于映射（字典或 Series）调整标签。

In [188]:
s

a    0.234266
b   -0.852730
c    1.764455
d    0.340539
e   -0.359231
dtype: float64

In [189]:
s.rename(str.upper)

A    0.234266
B   -0.852730
C    1.764455
D    0.340539
E   -0.359231
dtype: float64

如果调用的是函数，该函数在处理标签时，必须返回一个值，而且生成的必须是一组唯一值。此外，rename() 还可以调用字典或 Series。

In [190]:
df.rename(columns={'one': 'foo', 'two': 'bar'},
          index={'a': 'apple', 'b': 'banana', 'd': 'durian'})

,foo,bar,three
apple,-1.196879,0.578051,NaN
banana,-1.054653,-0.474366,0.191145
c,1.100068,-2.255187,1.916800
durian,NaN,1.087245,-0.480380


Pandas 不会重命名标签未包含在映射里的列或索引。注意，映射里多出的标签不会触发错误。

DataFrame.rename()还支持“轴式”习语，用这种方式可以指定单个 mapper，及执行映射的 axis。

In [191]:
df.rename({'one': 'foo', 'two': 'bar'}, axis='columns')

,foo,bar,three
a,-1.196879,0.578051,NaN
b,-1.054653,-0.474366,0.191145
c,1.100068,-2.255187,1.916800
d,NaN,1.087245,-0.480380


In [192]:
df.rename({'a': 'apple', 'b': 'banana', 'd': 'durian'}, axis='index')

,one,two,three
apple,-1.196879,0.578051,NaN
banana,-1.054653,-0.474366,0.191145
c,1.100068,-2.255187,1.916800
durian,NaN,1.087245,-0.480380


rename()方法还提供了 inplace 命名参数，默认为 False，并会复制底层数据。inplace=True 时，会直接在原数据上重命名。

rename()还支持用标量或列表更改 Series.name 属性。

In [193]:
s.rename("scalar-name")

a    0.234266
b   -0.852730
c    1.764455
d    0.340539
e   -0.359231
Name: scalar-name, dtype: float64

rename_axis()方法支持指定 多层索引 名称，与标签相对应。

In [195]:
df = pd.DataFrame({'x': [1, 2, 3, 4, 5, 6],
                   'y': [10, 20, 30, 40, 50, 60]},
                   index=pd.MultiIndex.from_product([['a', 'b', 'c'], [1, 2]],
                   names=['let', 'num']))

In [196]:
df

x   y
let num       
a   1    1  10
    2    2  20
b   1    3  30
    2    4  40
c   1    5  50
    2    6  60

In [197]:
df.rename_axis(index={'let': 'abc'})

x   y
abc num       
a   1    1  10
    2    2  20
b   1    3  30
    2    4  40
c   1    5  50
    2    6  60

In [198]:
df.rename_axis(index=str.upper)

x   y
LET NUM       
a   1    1  10
    2    2  20
b   1    3  30
    2    4  40
c   1    5  50
    2    6  60

## 迭代


Pandas 对象基于类型进行迭代操作。Series 迭代时被视为数组，基础迭代生成值。DataFrame 则遵循字典式习语，用对象的 key 实现迭代操作。
简言之，基础迭代（for i in object）生成：

- Series ：值
- DataFrame：列标签   

例如，DataFrame 迭代时输出列名：

In [199]:
df = pd.DataFrame({'col1': np.random.randn(3),
                   'col2': np.random.randn(3)}, index=['a', 'b', 'c'])

In [200]:
for col in df:
    print(col)

col1
col2


Pandas 对象还支持字典式的 items() (opens new window)方法，通过键值对迭代。

用下列方法可以迭代 DataFrame 里的行：
- iterrows()：把 DataFrame 里的行当作 （index， Series）对进行迭代。该操作把行转为 Series，同时改变数据类型，并对性能有影响。
- itertuples()：把 DataFrame 的行当作值的命名元组进行迭代。该操作比 iterrows() (opens new window)快的多，建议尽量用这种方法迭代 DataFrame 的值。

> **警告**   
> Pandas 对象迭代的速度较慢。大部分情况下，没必要对行执行迭代操作，建议用以下几种替代方式：   
>- 矢量化：很多操作可以用内置方法或 NumPy 函数，布尔索引……
>- 调用的函数不能在完整的 DataFrame / Series 上运行时，最好用 apply()，不要对值进行迭代操作。请参阅函数应用文档。
>- 如果必须对值进行迭代，请务必注意代码的性能，建议在 cython 或 numba 环境下实现内循环。参阅性能优化一节，查看这种操作方法的示例。

> **警告**   
> 永远不要修改迭代的内容，这种方式不能确保所有操作都能正常运作。基于数据类型，迭代器返回的是复制（copy）的结果，不是视图（view），这种写入可能不会生效！
> 下例中的赋值就不会生效：
> ```python
df = pd.DataFrame({'a': [1, 2, 3], 'b': ['a', 'b', 'c']})
for index, row in df.iterrows():
    row['a'] = 10
> ```



### 项目（items）
与字典型接口类似，items()通过键值对进行迭代：
- Series：（Index，标量值）对
- DataFrame：（列，Series）对

示例如下:

In [202]:
for label, ser in df.items():
    print(label)
    print(ser)

col1
a   -1.175399
b    1.448685
c   -0.018789
Name: col1, dtype: float64
col2
a   -1.578365
b   -2.140470
c   -0.145426
Name: col2, dtype: float64


### iterrows
iterrows()迭代 DataFrame 或 Series 里的每一行数据。这个操作返回一个迭代器，生成索引值及包含每行数据的 Series：

In [203]:
for row_index, row in df.iterrows():
    print(row_index, row, sep='\n')

a
col1   -1.175399
col2   -1.578365
Name: a, dtype: float64
b
col1    1.448685
col2   -2.140470
Name: b, dtype: float64
c
col1   -0.018789
col2   -0.145426
Name: c, dtype: float64


> 注意   
> iterrows()返回的是Series里的每一行数据，该操作不保留每行数据的数据类型，因为数据类型是通过DataFrame的列界定的。   
> 示例如下：  

In [205]:
df_orig = pd.DataFrame([[1, 1.5]], columns=['int', 'float'])
df_orig

,int,float
0,1,1.5


In [206]:
df_orig.dtypes

int        int64
float    float64
dtype: object

In [211]:
row = next(df_orig.iterrows())[1]
row

int      1.0
float    1.5
Name: 0, dtype: float64

row 里的值以 Series 形式返回，并被转换为浮点数，原始的整数值则在列 X：

In [212]:
row['int'].dtype

dtype('float64')

In [213]:
df_orig['int'].dtype

dtype('int64')

> 要想在行迭代时保存数据类型，最好用 itertuples()，这个函数返回值的命名元组，总的来说，该操作比iterrows()速度更快。

下例展示了怎样转置 DataFrame：

In [215]:
df2 = pd.DataFrame({'x': [1, 2, 3], 'y': [4, 5, 6]})
df2

,x,y
0,1,4
1,2,5
2,3,6


In [216]:
df2.T

,0,1,2
x,1,2,3
y,4,5,6


In [217]:
df2_t = pd.DataFrame({idx: values for idx, values in df2.iterrows()})
df2_t

,0,1,2
x,1,2,3
y,4,5,6


### itertuples

itertuples()方法返回为 DataFrame 里每行数据生成命名元组的迭代器。该元组的第一个元素是行的索引值，其余的值则是行的值。

In [218]:
for row in df.itertuples():
    print(row)

Pandas(Index='a', col1=-1.1753986036330775, col2=-1.5783654299002512)
Pandas(Index='b', col1=1.448685306375616, col2=-2.140470352461928)
Pandas(Index='c', col1=-0.018788948160827757, col2=-0.14542575199240806)


该方法不会把行转换为 Series，只是返回命名元组里的值。itertuples()保存值的数据类型，而且比 iterrows()快。

### .dt 访问器

Series 提供一个可以简单、快捷地返回 datetime 属性值的访问器。这个访问器返回的也是 Series，索引与现有的 Series 一样。

In [220]:
s = pd.Series(pd.date_range('20130101 09:10:12', periods=4))
s

0   2013-01-01 09:10:12
1   2013-01-02 09:10:12
2   2013-01-03 09:10:12
3   2013-01-04 09:10:12
dtype: datetime64[ns]

In [221]:
s.dt.hour

0    9
1    9
2    9
3    9
dtype: int64

In [222]:
s.dt.second

0    12
1    12
2    12
3    12
dtype: int64

In [223]:
s.dt.day

0    1
1    2
2    3
3    4
dtype: int64

用下列表达式进行筛选非常方便：

In [224]:
s[s.dt.day == 2]

1   2013-01-02 09:10:12
dtype: datetime64[ns]

时区转换也很轻松：


In [225]:
stz = s.dt.tz_localize('US/Eastern')
stz

0   2013-01-01 09:10:12-05:00
1   2013-01-02 09:10:12-05:00
2   2013-01-03 09:10:12-05:00
3   2013-01-04 09:10:12-05:00
dtype: datetime64[ns, US/Eastern]

In [226]:
stz.dt.tz

<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>

可以把这些操作连在一起：

In [227]:
s.dt.tz_localize('UTC').dt.tz_convert('US/Eastern')

0   2013-01-01 04:10:12-05:00
1   2013-01-02 04:10:12-05:00
2   2013-01-03 04:10:12-05:00
3   2013-01-04 04:10:12-05:00
dtype: datetime64[ns, US/Eastern]

还可以用 Series.dt.strftime()把 datetime 的值当成字符串进行格式化，支持与标准 strftime()同样的格式。

In [229]:
s = pd.Series(pd.date_range('20130101', periods=4))
s

0   2013-01-01
1   2013-01-02
2   2013-01-03
3   2013-01-04
dtype: datetime64[ns]

In [230]:
s.dt.strftime('%Y/%m/%d')

0    2013/01/01
1    2013/01/02
2    2013/01/03
3    2013/01/04
dtype: object

In [232]:
s = pd.Series(pd.period_range('20130101', periods=4))
s

0    2013-01-01
1    2013-01-02
2    2013-01-03
3    2013-01-04
dtype: period[D]

In [233]:
s.dt.strftime('%Y/%m/%d')

0    2013/01/01
1    2013/01/02
2    2013/01/03
3    2013/01/04
dtype: object

.dt 访问器还支持 period 与 timedelta。

In [235]:
s = pd.Series(pd.period_range('20130101', periods=4, freq='D'))
s

0    2013-01-01
1    2013-01-02
2    2013-01-03
3    2013-01-04
dtype: period[D]

In [236]:
s.dt.year

0    2013
1    2013
2    2013
3    2013
dtype: int64

In [237]:
s.dt.day

0    1
1    2
2    3
3    4
dtype: int64

In [239]:
s = pd.Series(pd.timedelta_range('1 day 00:00:05', periods=4, freq='s'))
s

0   1 days 00:00:05
1   1 days 00:00:06
2   1 days 00:00:07
3   1 days 00:00:08
dtype: timedelta64[ns]

In [240]:
s.dt.days

0    1
1    1
2    1
3    1
dtype: int64

In [241]:
s.dt.seconds

0    5
1    6
2    7
3    8
dtype: int64

In [242]:
s.dt.components

,days,hours,minutes,seconds,milliseconds,microseconds,nanoseconds
0,1,0,0,5,0,0,0
1,1,0,0,6,0,0,0
2,1,0,0,7,0,0,0
3,1,0,0,8,0,0,0


> 注意   
用这个访问器处理不是 datetime 类型的值时，Series.dt 会触发 TypeError 错误。

## 矢量化字符串方法

Series 支持字符串处理方法，可以非常方便地操作数组里的每个元素。这些方法会自动排除缺失值与空值，这也许是其最重要的特性。这些方法通过 Series 的 str 属性访问，一般情况下，这些操作的名称与内置的字符串方法一致。示例如下：

In [245]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca', np.nan, 'CABA', 'dog', 'cat'])
s.str.lower()

0       a
1       b
2       c
3    aaba
4    baca
5     NaN
6    caba
7     dog
8     cat
dtype: object

这里还提供了强大的模式匹配方法，但工业注意，模式匹配方法默认使用正则表达式。

参阅矢量化字符串方法，了解完整内容。

## 排序 
------
Pandas 支持三种排序方式，按索引标签排序，按列里的值排序，按两种方式混合排序。



### 按索引排序
Series.sort_index()与DataFrame.sort_index()方法用于按索引层级对 Pandas 对象排序。

In [246]:
df = pd.DataFrame({'one': pd.Series(np.random.randn(3), index=['a', 'b', 'c']),
                   'two': pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']),
                   'three': pd.Series(np.random.randn(3), index=['b', 'c', 'd'])})
df

,one,two,three
a,0.694471,-1.427734,NaN
b,-0.069566,-0.609766,-0.692887
c,0.015336,-0.495802,-0.185887
d,NaN,-0.648058,2.259051


In [248]:
unsorted_df = df.reindex(index=['a', 'd', 'c', 'b'],
                         columns=['three', 'two', 'one'])
unsorted_df

,three,two,one
a,NaN,-1.427734,0.694471
d,2.259051,-0.648058,NaN
c,-0.185887,-0.495802,0.015336
b,-0.692887,-0.609766,-0.069566


In [249]:
unsorted_df.sort_index()

,three,two,one
a,NaN,-1.427734,0.694471
b,-0.692887,-0.609766,-0.069566
c,-0.185887,-0.495802,0.015336
d,2.259051,-0.648058,NaN


In [250]:
unsorted_df.sort_index(ascending=False)

,three,two,one
d,2.259051,-0.648058,NaN
c,-0.185887,-0.495802,0.015336
b,-0.692887,-0.609766,-0.069566
a,NaN,-1.427734,0.694471


In [251]:
unsorted_df.sort_index(axis=1)

,one,three,two
a,0.694471,NaN,-1.427734
d,NaN,2.259051,-0.648058
c,0.015336,-0.185887,-0.495802
b,-0.069566,-0.692887,-0.609766


In [252]:
unsorted_df['three'].sort_index()

a         NaN
b   -0.692887
c   -0.185887
d    2.259051
Name: three, dtype: float64

### 按值排序
Series.sort_values()方法用于按值对 Series 排序。DataFrame.sort_values()方法用于按行列的值对 DataFrame 排序。DataFrame.sort_values()的可选参数 by 用于指定按哪列排序，该参数的值可以是一列或多列数据。

In [253]:
df1 = pd.DataFrame({'one': [2, 1, 1, 1],
                    'two': [1, 3, 2, 4],
                    'three': [5, 4, 3, 2]})

In [254]:
df1.sort_values(by='two')

,one,two,three
0,2,1,5
2,1,2,3
1,1,3,4
3,1,4,2


参数 by 支持列名列表，示例如下：

In [255]:
df1[['one', 'two', 'three']].sort_values(by=['one', 'two'])

,one,two,three
2,1,2,3
1,1,3,4
3,1,4,2
0,2,1,5


这些方法支持用 na_position 参数处理空值。

In [257]:
s[2] = np.nan
s

0       A
1       B
2     NaN
3    Aaba
4    Baca
5     NaN
6    CABA
7     dog
8     cat
dtype: object

In [258]:
s.sort_values()

0       A
3    Aaba
1       B
4    Baca
6    CABA
8     cat
7     dog
2     NaN
5     NaN
dtype: object

In [259]:
s.sort_values(na_position='first')

2     NaN
5     NaN
0       A
3    Aaba
1       B
4    Baca
6    CABA
8     cat
7     dog
dtype: object

### 按索引与值排序


通过参数 by 传递给 DataFrame.sort_values() (opens new window)的字符串可以引用列或索引层名。

In [261]:
idx = pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('a', 2),
                                 ('b', 2), ('b', 1), ('b', 1)])
idx

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 2),
            ('b', 2),
            ('b', 1),
            ('b', 1)],
           )

In [266]:
idx.names = ['first', 'second']
idx

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 2),
            ('b', 2),
            ('b', 1),
            ('b', 1)],
           names=['first', 'second'])

In [267]:
df_multi = pd.DataFrame({'A': np.arange(6, 0, -1)}, index=idx)
df_multi

A
first second   
a     1       6
      2       5
      2       4
b     2       3
      1       2
      1       1

In [268]:
df_multi.sort_values(by=['second', 'A'])

A
first second   
b     1       1
      1       2
a     1       6
b     2       3
a     2       4
      2       5

> **注意**   
字符串、列名、索引层名重名时，会触发警告提示，并以列名为准。后期版本中，这种情况将会触发模糊错误。

### 搜索排序
Series 支持 searchsorted()方法，这与numpy.ndarray.searchsorted()的操作方式类似。

In [274]:
ser = pd.Series([1, 2, 3])
ser

0    1
1    2
2    3
dtype: int64

In [270]:
ser.searchsorted([0, 3])

array([0, 2])

In [271]:
ser.searchsorted([0, 4])

array([0, 3])

In [272]:
ser.searchsorted([1, 3], side='right')

array([1, 3])

In [273]:
ser.searchsorted([1, 3], side='left')

array([0, 2])

In [276]:
ser = pd.Series([3, 1, 2])
ser

0    3
1    1
2    2
dtype: int64

In [277]:
ser.searchsorted([0, 3], sorter=np.argsort(ser))

array([0, 2])

### 最大值与最小值
Series 支持 nsmallest()与 nlargest()方法，本方法返回 N 个最大或最小的值。对于数据量大的 Series 来说，该方法比先为整个 Series 排序，再调用 head(n) 这种方式的速度要快得多。

In [278]:
s = pd.Series(np.random.permutation(10))
s

0    0
1    2
2    1
3    5
4    6
5    8
6    9
7    3
8    7
9    4
dtype: int64

In [279]:
s.sort_values()

0    0
2    1
1    2
7    3
9    4
3    5
4    6
8    7
5    8
6    9
dtype: int64

In [280]:
s.nsmallest(3)

0    0
2    1
1    2
dtype: int64

In [281]:
s.nlargest(3)

6    9
5    8
8    7
dtype: int64

DataFrame 也支持 nlargest 与 nsmallest 方法。

In [284]:
df = pd.DataFrame({'a': [-2, -1, 1, 10, 8, 11, -1],
                   'b': list('abdceff'),
                   'c': [1.0, 2.0, 4.0, 3.2, np.nan, 3.0, 4.0]})
df

,a,b,c
0,-2,a,1.0
1,-1,b,2.0
2,1,d,4.0
3,10,c,3.2
4,8,e,NaN
5,11,f,3.0
6,-1,f,4.0


In [285]:
df.nlargest(3, 'a')

,a,b,c
5,11,f,3.0
3,10,c,3.2
4,8,e,NaN


In [286]:
df.nlargest(5, ['a', 'c'])

,a,b,c
5,11,f,3.0
3,10,c,3.2
4,8,e,NaN
2,1,d,4.0
6,-1,f,4.0


In [287]:
df.nsmallest(5, ['a', 'c'])

,a,b,c
0,-2,a,1.0
1,-1,b,2.0
6,-1,f,4.0
2,1,d,4.0
4,8,e,NaN


### 用多层索引的列排序
列为多层索引时，可以显式排序，用 by 指定所有层级。

In [289]:
df1.columns = pd.MultiIndex.from_tuples([('a', 'one'),
                                         ('a', 'two'),
                                         ('b', 'three')])
df1

a         b
  one two three
0   2   1     5
1   1   3     4
2   1   2     3
3   1   4     2

In [290]:
df1.sort_values(by=('a', 'two'))

a         b
  one two three
0   2   1     5
2   1   2     3
1   1   3     4
3   1   4     2

In [334]:
res_pd = pd.read_csv('isbgphysical1.csv', index_col=0)
res_pd

,NEname,dateTime,SC.AnsSessionTerm.fromVoLTE,SC.OrigRelBeforeRing.fromVoLTE,SC.OrigRelAfterRing.fromVoLTE,SC.FailSessionOrig.403.fromVoLTE,SC.FailSessionOrig.404.fromVoLTE,SC.FailSessionOrig.408.fromVoLTE,SC.FailSessionOrig.480.fromVoLTE,SC.FailSessionOrig.484.fromVoLTE,...,SC.FailSessionTerm.484.fromVoLTE,SC.FailSessionTerm.486.fromVoLTE,SC.FailSessionTerm.487.fromVoLTE,SC.FailSessionTerm.600.fromVoLTE,SC.FailSessionTerm.603.fromVoLTE,SC.FailSessionTerm.604.fromVoLTE,SC.TermRelBeforeRing.fromVoLTE,SC.TermRelAfterRing.fromVoLTE,SC.AttSessionOrig.fromVoLTE,SC.AttSessionTerm.fromVoLTE
0,CQISBG6BZX,20210124141500,53813.0,5575.0,12508.0,2.0,3.0,1677.0,383.0,0.0,...,0.0,741.0,1542.0,0.0,0.0,0.0,3065.0,11110.0,72288.0,74430.0
1,CQISBG8BZX,20210124141500,53866.0,5530.0,12597.0,5.0,7.0,1630.0,351.0,0.0,...,0.0,702.0,1551.0,0.0,0.0,0.0,2966.0,10783.0,71794.0,73911.0
2,CQISBG7BZX,20210124141500,54009.0,5552.0,12525.0,7.0,5.0,1603.0,384.0,0.0,...,0.0,726.0,1530.0,0.0,0.0,0.0,3059.0,10939.0,71579.0,74462.0
3,CQISBG9BZX,20210124141500,54549.0,5453.0,12484.0,9.0,6.0,1599.0,401.0,0.0,...,0.0,739.0,1546.0,0.0,1.0,0.0,3025.0,10934.0,71822.0,75027.0
4,CQISBG5BZX,20210124141500,53911.0,5472.0,12285.0,3.0,4.0,1679.0,373.0,0.0,...,0.0,734.0,1560.0,0.0,0.0,0.0,2867.0,10743.0,71177.0,73957.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,CQISBG13BHW,20210125170000,141488.0,13974.0,29136.0,26.0,16.0,2433.0,1880.0,0.0,...,0.0,2386.0,1533.0,0.0,160.0,3.0,7786.0,36085.0,173031.0,195096.0
677,CQISBG11BHW,20210125171500,133998.0,14081.0,29326.0,8.0,3.0,2240.0,1906.0,0.0,...,0.0,2277.0,1190.0,0.0,151.0,2.0,7156.0,33611.0,174115.0,183485.0
678,CQISBG12BHW,20210125171500,134338.0,14084.0,29888.0,24.0,5.0,2476.0,1888.0,0.0,...,0.0,2589.0,1349.0,0.0,102.0,0.0,7225.0,33571.0,175596.0,184315.0
679,CQISBG14BHW,20210125171500,133933.0,13820.0,30258.0,20.0,6.0,2516.0,1930.0,0.0,...,0.0,2340.0,1420.0,0.0,117.0,2.0,7172.0,33732.0,175660.0,183737.0


In [341]:
res_pd.groupby('NEname').groups.keys()

dict_keys(['CQISBG11BHW', 'CQISBG12BHW', 'CQISBG13BHW', 'CQISBG14BHW', 'CQISBG5BZX', 'CQISBG6BZX', 'CQISBG7BZX', 'CQISBG8BZX', 'CQISBG9BZX'])

In [293]:
time_list = [20210124141500]
nes = r'^CQISBG\d{1,2}BZX'

In [298]:
res_pd = res_pd[res_pd['NEname'].str.contains(nes) & res_pd['dateTime'].isin(time_list)]
res_pd

,NEname,dateTime,SC.AnsSessionTerm.fromVoLTE,SC.OrigRelBeforeRing.fromVoLTE,SC.OrigRelAfterRing.fromVoLTE,SC.FailSessionOrig.403.fromVoLTE,SC.FailSessionOrig.404.fromVoLTE,SC.FailSessionOrig.408.fromVoLTE,SC.FailSessionOrig.480.fromVoLTE,SC.FailSessionOrig.484.fromVoLTE,...,SC.FailSessionTerm.484.fromVoLTE,SC.FailSessionTerm.486.fromVoLTE,SC.FailSessionTerm.487.fromVoLTE,SC.FailSessionTerm.600.fromVoLTE,SC.FailSessionTerm.603.fromVoLTE,SC.FailSessionTerm.604.fromVoLTE,SC.TermRelBeforeRing.fromVoLTE,SC.TermRelAfterRing.fromVoLTE,SC.AttSessionOrig.fromVoLTE,SC.AttSessionTerm.fromVoLTE
0,CQISBG6BZX,20210124141500,53813.0,5575.0,12508.0,2.0,3.0,1677.0,383.0,0.0,...,0.0,741.0,1542.0,0.0,0.0,0.0,3065.0,11110.0,72288.0,74430.0
1,CQISBG8BZX,20210124141500,53866.0,5530.0,12597.0,5.0,7.0,1630.0,351.0,0.0,...,0.0,702.0,1551.0,0.0,0.0,0.0,2966.0,10783.0,71794.0,73911.0
2,CQISBG7BZX,20210124141500,54009.0,5552.0,12525.0,7.0,5.0,1603.0,384.0,0.0,...,0.0,726.0,1530.0,0.0,0.0,0.0,3059.0,10939.0,71579.0,74462.0
3,CQISBG9BZX,20210124141500,54549.0,5453.0,12484.0,9.0,6.0,1599.0,401.0,0.0,...,0.0,739.0,1546.0,0.0,1.0,0.0,3025.0,10934.0,71822.0,75027.0
4,CQISBG5BZX,20210124141500,53911.0,5472.0,12285.0,3.0,4.0,1679.0,373.0,0.0,...,0.0,734.0,1560.0,0.0,0.0,0.0,2867.0,10743.0,71177.0,73957.0


In [320]:
name_dict = res_pd.groupby('NEname').groups
name_dict

{'CQISBG5BZX': [4], 'CQISBG6BZX': [0], 'CQISBG7BZX': [2], 'CQISBG8BZX': [1], 'CQISBG9BZX': [3]}

In [318]:
res_pd.loc[[4,0], ['dateTime', 'SC.AnsSessionTerm.fromVoLTE']]

,dateTime,SC.AnsSessionTerm.fromVoLTE
4,20210124141500,53911.0
0,20210124141500,53813.0


In [333]:
res_pd['dateTime'].to_list()

[20210124141500,
 20210124141500,
 20210124141500,
 20210124141500,
 20210124141500]

In [321]:
err_fullnames = [
    'SC.AnsSessionTerm.fromVoLTE',
    'SC.OrigRelBeforeRing.fromVoLTE',
    'SC.OrigRelAfterRing.fromVoLTE',
    'SC.FailSessionOrig.403.fromVoLTE',
    'SC.FailSessionOrig.404.fromVoLTE',
    'SC.FailSessionOrig.408.fromVoLTE',
    'SC.FailSessionOrig.480.fromVoLTE',
    'SC.FailSessionOrig.484.fromVoLTE',
    'SC.FailSessionOrig.487.fromVoLTE',
    'SC.FailSessionOrig.486.fromVoLTE',                               
    'SC.FailSessionOrig.600.fromVoLTE',
    'SC.FailSessionOrig.603.fromVoLTE',
    'SC.FailSessionOrig.604.fromVoLTE',
    'SC.AnsSessionOrig.fromVoLTE',
    'SC.FailSessionTerm.403.fromVoLTE',
    'SC.FailSessionTerm.404.fromVoLTE',
    'SC.FailSessionTerm.408.fromVoLTE',
    'SC.FailSessionTerm.480.fromVoLTE',
    'SC.FailSessionTerm.484.fromVoLTE',
    'SC.FailSessionTerm.486.fromVoLTE',
    'SC.FailSessionTerm.487.fromVoLTE',
    'SC.FailSessionTerm.600.fromVoLTE',
    'SC.FailSessionTerm.603.fromVoLTE',
    'SC.FailSessionTerm.604.fromVoLTE',
    'SC.TermRelBeforeRing.fromVoLTE',
    'SC.TermRelAfterRing.fromVoLTE',
    'SC.AttSessionOrig.fromVoLTE',
    'SC.AttSessionTerm.fromVoLTE',
]

In [327]:
err_se = pd.Series(err_fullnames)
err_se

0          SC.AnsSessionTerm.fromVoLTE
1       SC.OrigRelBeforeRing.fromVoLTE
2        SC.OrigRelAfterRing.fromVoLTE
3     SC.FailSessionOrig.403.fromVoLTE
4     SC.FailSessionOrig.404.fromVoLTE
5     SC.FailSessionOrig.408.fromVoLTE
6     SC.FailSessionOrig.480.fromVoLTE
7     SC.FailSessionOrig.484.fromVoLTE
8     SC.FailSessionOrig.487.fromVoLTE
9     SC.FailSessionOrig.486.fromVoLTE
10    SC.FailSessionOrig.600.fromVoLTE
11    SC.FailSessionOrig.603.fromVoLTE
12    SC.FailSessionOrig.604.fromVoLTE
13         SC.AnsSessionOrig.fromVoLTE
14    SC.FailSessionTerm.403.fromVoLTE
15    SC.FailSessionTerm.404.fromVoLTE
16    SC.FailSessionTerm.408.fromVoLTE
17    SC.FailSessionTerm.480.fromVoLTE
18    SC.FailSessionTerm.484.fromVoLTE
19    SC.FailSessionTerm.486.fromVoLTE
20    SC.FailSessionTerm.487.fromVoLTE
21    SC.FailSessionTerm.600.fromVoLTE
22    SC.FailSessionTerm.603.fromVoLTE
23    SC.FailSessionTerm.604.fromVoLTE
24      SC.TermRelBeforeRing.fromVoLTE
25       SC.TermRelAfterR

In [331]:
err_list = err_se[err_se.str.contains('404', case=False)].to_list()
err_list

['SC.FailSessionOrig.404.fromVoLTE', 'SC.FailSessionTerm.404.fromVoLTE']

In [344]:
s1 = pd.Series([True, False, False, False, False, False, False, False])
s2 = pd.Series([False, True, False, False, False, False, False, False])
s1 | s2

0     True
1     True
2    False
3    False
4    False
5    False
6    False
7    False
dtype: bool

In [346]:
res_pd['NEname'].str.contains(nes, case=False)

0       True
1       True
2       True
3       True
4       True
       ...  
676    False
677    False
678    False
679    False
680    False
Name: NEname, Length: 681, dtype: bool

In [350]:
ne = nes
times_df = pd.DataFrame(time_list, columns=['dateTime'])

temp_df = res_pd[res_pd['NEname'].str.contains(ne, case=False) & res_pd['dateTime'].isin(time_list)]
temp_df = temp_df.groupby('dateTime').sum()
temp_df = temp_df.merge(times_df, how='right', on='dateTime')
temp_df['NEname'] = ne
temp_df

,dateTime,SC.AnsSessionTerm.fromVoLTE,SC.OrigRelBeforeRing.fromVoLTE,SC.OrigRelAfterRing.fromVoLTE,SC.FailSessionOrig.403.fromVoLTE,SC.FailSessionOrig.404.fromVoLTE,SC.FailSessionOrig.408.fromVoLTE,SC.FailSessionOrig.480.fromVoLTE,SC.FailSessionOrig.484.fromVoLTE,SC.FailSessionOrig.487.fromVoLTE,...,SC.FailSessionTerm.486.fromVoLTE,SC.FailSessionTerm.487.fromVoLTE,SC.FailSessionTerm.600.fromVoLTE,SC.FailSessionTerm.603.fromVoLTE,SC.FailSessionTerm.604.fromVoLTE,SC.TermRelBeforeRing.fromVoLTE,SC.TermRelAfterRing.fromVoLTE,SC.AttSessionOrig.fromVoLTE,SC.AttSessionTerm.fromVoLTE,NEname
0,20210124141500,270148.0,27582.0,62399.0,26.0,25.0,8188.0,1892.0,0.0,2885.0,...,3642.0,7729.0,0.0,1.0,0.0,14982.0,54509.0,358660.0,371787.0,"^CQISBG\d{1,2}BZX"
